In [1]:
import pandas as pd
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
class lstm(nn.Module):
    def __init__(self,num):
        super().__init__()
        self.lstm = nn.LSTM(1,20,2,dropout=0.1,batch_first=True)
        self.linear = nn.Linear(20,5)
        self.relu=nn.ReLU()
        self.linear1 = nn.Linear(5,1)
        self.num = num
        self.lstm1 = nn.LSTM(1,20,2,dropout=0.2,batch_first=True)
    def forward(self,x):
        #print(x.shape)
        x = x.unsqueeze(2)
        output, (hn, cn) = self.lstm(x)
        output1=[]
        
        for i in range(self.num):
            output, (hn, cn) = self.lstm1(x[:,-1,:].unsqueeze(1),(hn,cn))
            out = self.relu(self.linear(output[:,-1,:]))
            out1 = self.linear1(out)
            if i <=self.num-1:
                out1 = out1.unsqueeze(2)
                output1.append(out1)
                #print(out1.shape)
                x = torch.cat([x,out1],dim=1)
                output, (hn, cn) = self.lstm(x[:,1:,:])
        return torch.cat(output1,dim=0)

In [3]:
def data_process(data1):
        num = len(data1)-5
        data = np.ones((num,5))
        label = np.ones((num,2))
        #data1 = data1/np.max(data1)
        for i in range(num):
            #print(data1[i:i+5].shape)
            data[i,:]=data1[i:i+5]
            label[i,:]=data1[i+5:i+7]
        train_x,test_x,train_y,test_y = train_test_split(data,label,test_size=0.2,random_state=1,shuffle=True)
        #print(train_x)
        #print(train_y)
        np.save('train_xdata.npy',train_x)
        np.save('train_ydata.npy',train_y)
        np.save('test_xdata.npy',test_x)
        np.save('test_ydata.npy',test_y)

In [4]:
data = pd.read_csv('result_eva5.csv')
data = data['dcp'].to_numpy()
data_process(data)

In [5]:
class CustomDataset(Dataset):
    def __init__(self, annotations_file,annotations_file_y):
        self.x = np.load(annotations_file)
        self.y = np.load(annotations_file_y)
    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        input_x = self.x[idx]
        label =self.y[idx]
        return torch.from_numpy(input_x), torch.from_numpy(label)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
model = lstm(2)
model.to(device)
traindata = CustomDataset('train_xdata.npy','train_ydata.npy')
valdata = CustomDataset('test_xdata.npy','test_ydata.npy')

In [8]:
train_loader = DataLoader(traindata,batch_size=16,shuffle=True)
val_loader = DataLoader(traindata,batch_size=8,shuffle=False)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X=X.float()
        y=y.float()
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        pred = torch.flatten(pred)
        #pred = output[:,-2:]
        y = torch.flatten(y)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X=X.float()
            y=y.float()
            X, y = X.to(device), y.to(device)
            pred = model(X)
            #pred = output[:,-2:]
            pred = torch.flatten(pred)
            #pred = output[:,-2:]
            y = torch.flatten(y)
            test_loss += loss_fn(pred, y).item()
    test_loss /= num_batches
    print(f"Test Error: \n , Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [12]:
epochs = 1000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(val_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 490.631897  [    0/   71]
Test Error: 
 , Avg loss: 543.996568 

Epoch 2
-------------------------------
loss: 572.921509  [    0/   71]
Test Error: 
 , Avg loss: 543.984806 

Epoch 3
-------------------------------
loss: 459.687744  [    0/   71]
Test Error: 
 , Avg loss: 543.969852 

Epoch 4
-------------------------------
loss: 406.092651  [    0/   71]
Test Error: 
 , Avg loss: 543.952287 

Epoch 5
-------------------------------
loss: 435.145874  [    0/   71]
Test Error: 
 , Avg loss: 543.930142 

Epoch 6
-------------------------------
loss: 536.981689  [    0/   71]
Test Error: 
 , Avg loss: 543.901523 

Epoch 7
-------------------------------
loss: 498.207947  [    0/   71]
Test Error: 
 , Avg loss: 543.863949 

Epoch 8
-------------------------------
loss: 650.984558  [    0/   71]
Test Error: 
 , Avg loss: 543.811839 

Epoch 9
-------------------------------
loss: 500.816498  [    0/   71]
Test Error: 
 , Avg loss: 543.746284 

E

In [13]:
x = torch.tensor([1, 2, 3, 4])
x.unsqueeze(1)

tensor([[1],
        [2],
        [3],
        [4]])

In [14]:
np.max(data)

2738.326693227093

In [15]:
torch.save(model.state_dict(),'backward_model.pt')